In [1]:
"""Package Loading"""
"""General"""
from webscraping_dictionaries import *
from webscraping_functions import *
#from email.mime.text import MIMEText
#from email.mime.multipart import MIMEMultipart
"""Chrome Options"""
#from selenium.webdriver import Chrome 
#from selenium.webdriver.chrome.service import Service 
#from webdriver_manager.chrome import ChromeDriverManager

"""Firefox Options"""
#from selenium.webdriver import Firefox
#from selenium.webdriver.firefox.service import Service
#from webdriver_manager.firefox import GeckoDriverManager


"""Initialize the driver"""

"""Driver Setup"""

#following steps from here: https://www.zenrows.com/blog/scraping-javascript-rendered-web-pages#installing-the-requirements
#critical help using the click method from here https://www.geeksforgeeks.org/click-element-method-selenium-python/

"""Chrome Version"""
#options = webdriver.ChromeOptions()
#options.headless = True
#options.page_load_strategy = 'none' 
#chrome_path = ChromeDriverManager().install()
#chrome_service = Service(chrome_path)
#driver = Chrome(options=options, service=chrome_service) 
#driver.implicitly_wait(5)

"""Firefox Version"""  
#options = webdriver.FirefoxOptions()
#options.headless = True
#options.add_argument("--headless=new")
#options.page_load_strategy = 'none' 
#firefox_path = GeckoDriverManager().install() 
#firefox_service = Service(firefox_path)
#driver = Firefox(options=options, service=firefox_service)
#driver.implicitly_wait(5)

'Firefox Version'

"""DEBUGGING SANDBOX"""

In [ ]:
#table this for now, since we still want to check for comp plan and ordinance keywords anyway
"Charlotte County Solar Development Page"
def charlotte_county_solar_development():
    driver.get("https://www.charlottecountyva.gov/departments/planning___zoning/solar_development.php")
    time.sleep(1)
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")


In [ ]:
New_Alerts = []

"Run all the webscraping functions, single thread version"
for locality in agendacenter_dictionary:
    try:
        alert = agendacenter(locality)
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item + " using AgendaCenter code."
        New_Alerts.append(error_alert)
        continue
            
for item in boarddocs_dictionary:
    try:
        alert = boarddocs(boarddocs_dictionary[item][0],boarddocs_dictionary[item][1],boarddocs_dictionary[item][2])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item + " using BoardDocs code."
        New_Alerts.append("\n" + error_alert)
        continue

for item in civicclerk_dictionary:
    try:
        alert=civicclerk(civicclerk_dictionary[item][0],civicclerk_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using CivicClerk code"
        New_Alerts.append(error_alert)
        continue
        
for item in civicweb_dictionary:
    try:
        alert=civicweb(civicweb_dictionary[item][0],civicweb_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using CivicWeb code"
        New_Alerts.append(error_alert)
        continue
        
for item in document_center_dictionary:
    try:
        alert=document_center(document_center_dictionary[item][0],document_center_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using Document Center code"
        New_Alerts.append(error_alert)
        continue

for item in escribe_dictionary:
    try:
        alert=escribe(escribe_dictionary[item][0],escribe_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using EScribe code"
        New_Alerts.append(error_alert)
        continue

for item in granicus_dictionary:
    try:
        alert = granicus(granicus_dictionary[item][0],granicus_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item + " using Granicus code"
        New_Alerts.append(error_alert)
        continue
        
for item in granicus_2_dictionary:
    try:
        alert = granicus_version_2(granicus_2_dictionary[item][0],granicus_2_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item + " using Granicus Version 2 code"
        New_Alerts.append(error_alert)
        continue

for item in laserfiche_dictionary:
    try:
        alert=laserfiche(laserfiche_dictionary[item][0],laserfiche_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using LaserFiche code"
        New_Alerts.append(error_alert)
        continue

for item in legistar_dictionary:
    try:
        alert=legistar(legistar_dictionary[item][0],legistar_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using Legistar code"
        New_Alerts.append(error_alert)
        continue

for item in meetingstable_dictionary:
    try:
        alert=meetings_table(meetingstable_dictionary[item][0],meetingstable_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using MeetingsTable code"
        New_Alerts.append(error_alert)
        continue

for item in novusagenda_dictionary:
    try:
        alert=novusagenda(novusagenda_dictionary[item][0],novusagenda_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using NovusAGENDA code"
        New_Alerts.append(error_alert)
        continue

for item in onbase_dictionary:
    try:
        alert=onbase(onbase_dictionary[item][0],onbase_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using OnBase code"
        New_Alerts.append(error_alert)
        continue

for item in primegov_dictionary:
    try:
        alert=prime_gov(primegov_dictionary[item][0],primegov_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using PrimeGov code"
        New_Alerts.append(error_alert)
        continue

for item in php_table_dictionary:
    try:
        alert=php_table(php_table_dictionary[item][0],php_table_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using PHP Table code"
        New_Alerts.append(error_alert)
        continue

for item in county_dictionary_single_variable:
    try:
        alert=county_dictionary_single_variable[item][0](county_dictionary_single_variable[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item
        New_Alerts.append(error_alert)
        continue

for item in county_dictionary_double_variable:
    try:
        alert=county_dictionary_double_variable[item][0](county_dictionary_double_variable[item][1],county_dictionary_double_variable[item][2])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item
        New_Alerts.append(error_alert)
        continue

for item in city_dictionary_single_variable:
    try:
        alert=city_dictionary_single_variable[item][0](city_dictionary_single_variable[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item
        New_Alerts.append(error_alert)
        continue

for item in city_dictionary_double_variable:
    try:
        alert=city_dictionary_double_variable[item][0](city_dictionary_double_variable[item][1],city_dictionary_double_variable[item][2])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item
        New_Alerts.append(error_alert)
        continue

In [ ]:
for item in New_Alerts:
    print(item)

In [ ]:
#empty list 
Solar_Alerts = []
Error_Alerts = []
Unreadable_File_Alerts = []
Other_Alerts = []

#loop that checks if the word in the meeting agenda is in the list of keywords...if that happens, append to empty list
for message in New_Alerts:
    if "Solar" in message:
        Solar_Alerts.append(message)
    elif "Error" in message or "Not Reachable" in message:
        Error_Alerts.append(message)
    elif "scanned" in message:
        Unreadable_File_Alerts.append(message)
    else:
        Other_Alerts.append(message)

# Convert categorized alerts to strings
solar_alerts_str = ", \n".join(Solar_Alerts)
error_alerts_str = ", \n".join(Error_Alerts)
unread_alerts_str = ", \n".join(Unreadable_File_Alerts)
other_alerts_str = ", \n".join(Other_Alerts)


In [ ]:
alerts = ("Solar Alerts:\n" + solar_alerts_str + "\n\nUnreadable Alerts: \n" + unread_alerts_str + "\n\nError Alerts:\n" + error_alerts_str + "\n\nOther Alerts:\n" + other_alerts_str)

In [ ]:
print(alerts)

In [ ]:
#come back to this another day
all_dictionaries=[agendacenter_dictionary,boarddocs_dictionary,civicclerk_dictionary,civicweb_dictionary,document_center_dictionary,escribe_dictionary,granicus_dictionary,granicus_2_dictionary,laserfiche_dictionary,legistar_dictionary,meetingstable_dictionary,novusagenda_dictionary,php_table_dictionary,primegov_dictionary,county_dictionary_single_variable,county_dictionary_double_variable,city_dictionary_single_variable,city_dictionary_double_variable]

for dictionary in all_dictionaries:
    for entry in dictionary:
        print(entry)

In [ ]:
#reconfiguring how the scraping methods work
#leverage object oriented programming and the possibilities of functions
#currently, each website has its own custom function, and the repeated document sharing websites have their own customized function as well
#few repeated functions despite potential for more, and for better generalization
#inconsistent methodoloy that evolved over time as I kept remaking the functions from scratch
#potential to take the common steps and turn those into functions rather than having them rewritten dozens of times
#opportunities to keep optimizing

In [ ]:
"""AgendaCenter"""
def agendacenter(locality):
    driver.get(agendacenter_dictionary[locality][0])
    time.sleep(5)
    table_rows = driver.find_elements(By.CSS_SELECTOR, meetings_tags['agendacenter'])
    future_meetings = [item for item in table_rows if check_meeting_date(search_dates(item.text,languages=['en'])[0][0])==True]
    agenda_links = [item.find_elements(By.CSS_SELECTOR,"a")[1].get_attribute("href") for item in future_meetings]
    messages = search_pdf(agenda_links,locality)
    return messages

In [5]:
driver.get("https://www.waynesboro.va.us/AgendaCenter/Search/?term=&CIDs=7,1,4,&startDate=&endDate=&dateRange=&dateSelector=")

In [6]:
table_rows = driver.find_elements(By.CSS_SELECTOR, meetings_tags['agendacenter'])

In [176]:
check_meeting_date(search_dates(table_rows[0].text,languages=['en'])[0][0])

False

In [177]:
future_meetings = [item for item in table_rows if check_meeting_date(search_dates(item.text,languages=['en'])[0][0])==True]

In [178]:
agenda_links = [item.find_elements(By.CSS_SELECTOR,"a")[1].get_attribute("href") for item in future_meetings]

In [179]:
agenda_links

[]

In [172]:
search_pdf([agenda_links[0]],"City of Suffolk")

['Keyword(s) Comprehensive Plan, Solar found in upcoming meeting for City of Suffolk. https://www.suffolkva.us/AgendaCenter/ViewFile/Agenda/_12172024-3425']

In [ ]:
#AgendaCenter Strategy:
"""
* = I should make this a function

1: *navigate to the locality archives url
2: *wait for the page to load enough for the desired content to appear
3: create the empty messages list
4: locate the rows of meeting titles
5: filter for meetings happening in the future
    5a: two common string formats with AgendaCenter, need to split the date string or it throws an error (search for a package or class or function that does this automatically)
6: collect the links for the agendas for future meetings out of the meeting information rows
7: go through each agenda link 
    7a: *navigate to the agenda url
    7b: *wait for the content to load
    7c: collect the content tags (the page-by-page PDF search for content that loads on scroll should be part of this)
        7c1: if the textLayer tag doesn't exist, it's probably a web page not a PDF, and collect this commonly used webpage content tag instead
    7d: for a PDF, check that there is text in the content tags, otherwise it's likely an unsearchable scan
        7d1: if there is text in the content tags, search the text for the specified keywords
        7d2: if the search result is not empty, add an alert message and provide the agenda url
    7e: if there is no text in the content tags, provide an "unreadable" messaage and a link to the agenda for manual review
8: return any alert messages
"""


